In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import re
import spacy 
from PIL import Image
from wordcloud import WordCloud
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
col = ["File_Name","Auteur", "Texte"]
df= pd.DataFrame(columns=col)
print(df)

Empty DataFrame
Columns: [File_Name, Auteur, Texte]
Index: []


In [3]:
chemin= os.getcwd() + '\\training_data\\'
chemin=str(chemin)

In [4]:
datas=[]
file_dir=[]
file_name=[]

for file in os.listdir(chemin):
    file_name.append(file)
    file_dir= chemin+str(file)
    with open(file_dir, 'r') as f:
        data = f.read()
        datas.append(data)
        f.close()
print(datas)
print(file_name)
df=pd.DataFrame({"Texte":[], "File_Name":[]})


['Idris was well content with this resolve of mine.\n', 'I was faint, even fainter than the hateful modernity of that accursed city had made me.\n', 'Above all, I burn to know the incidents of your own passage through the dark Valley and Shadow.\n', 'He might see, perhaps, one or two points with unusual clearness, but in so doing he, necessarily, lost sight of the matter as a whole.\n', 'All obeyed the Lord Protector of dying England; all looked up to him.\n', 'It was difficult, at any given time, to say with certainty upon which of its two stories one happened to be.\n', 'Now, it cannot be maintained, even, that by the crawling system the greatest amount of truth would be attained in any long series of ages, for the repression of imagination was an evil not to be compensated for by any superior certainty in the ancient modes of investigation.\n', 'Alien it indeed was to all art and literature which sane and balanced readers know, but we recognised it as the thing hinted of in the forb

In [5]:
df["Texte"]=datas
df["File_Name"]=file_name
print(df)


                                                  Texte           File_Name
0     Idris was well content with this resolve of mi...  doc_id00001MWS.txt
1     I was faint, even fainter than the hateful mod...  doc_id00002HPL.txt
2     Above all, I burn to know the incidents of you...  doc_id00003EAP.txt
3     He might see, perhaps, one or two points with ...  doc_id00004EAP.txt
4     All obeyed the Lord Protector of dying England...  doc_id00005MWS.txt
...                                                 ...                 ...
3595  My countenance was painted with the hues of il...  doc_id05087MWS.txt
3596  It must have been merely the association of an...  doc_id05088HPL.txt
3597  Dangers he met unflinchingly; crimes he commit...  doc_id05089HPL.txt
3598  As this state of wretchedness became more conf...  doc_id05090MWS.txt
3599  Were the pride of ancestry, the patrician spir...  doc_id05092MWS.txt

[3600 rows x 2 columns]


In [6]:
pattern= "[A-Z]{3}[.]+txt$"
search = []    
for values in df["File_Name"]:
    search.append(re.search(r'[A-Z]{3}', values).group())
df['Auteur'] = search


In [7]:
#Modèle entrainé le plus léger et suffisant ici. Regarder si md ou lrg à meilleur ratio time_process/accurancy
nlp = spacy.load("en_core_web_sm")


In [8]:
#Code plus efficient afin de tokeniser, lemmatiser et rajouter les POS. 
# Ne pas oublier de rajouter les stopwords!!!!!
tokens = []
lemma = []
pos = []
stop_words= []
ent= []

for doc in nlp.pipe(df['Texte'].astype('unicode').values, batch_size=50, n_threads=3):
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
        stop_words.append([n.text for n in doc if not n.is_stop])
        ent.append([e.label_ for e in doc.ents])

    else:
        # Ajouter des blancs si erreur pour avoir le même nombre d'entrées
        lemma.append(None)
        pos.append(None)
        stop_words.append(None)
        ent.append(None)

df['Tokens'] = tokens
df['Tokens_NoStopW']= stop_words
df['lemma'] = lemma
df['PartOfSpeech'] = pos
df['ent'] = ent

DEBUT DE L'EXPLORATION

In [9]:
auteur = df.groupby("Auteur")
auteur.describe().head()


Texte                                                                 \
       count unique                                                top freq   
Auteur                                                                        
EAP     1432   1432  When it reached the house it domesticated itse...    1   
HPL     1024   1024  It is of this loot in particular that I must n...    1   
MWS     1144   1144  I shall not pause in my exertions until that t...    1   

       File_Name                                 Tokens         ...  \
           count unique                 top freq  count unique  ...   
Auteur                                                          ...   
EAP         1432   1432  doc_id03686EAP.txt    1   1432   1432  ...   
HPL         1024   1024  doc_id00022HPL.txt    1   1024   1024  ...   
MWS         1144   1144  doc_id02012MWS.txt    1   1144   1144  ...   

                                                    lemma      PartOfSpeech  \
                                                      top freq        count   
Auteur                                                                        
EAP     [as, the, evening, wear, away, -PRON-, become,...    1         1432   
HPL     [if, this, man, could, not, be, restore, to, l...    1         1024   
MWS     [-PRON-, believe, that, without, the, slight, ...    1         1144   

                                                                        ent  \
       unique                                                top freq count   
Auteur                                                                        
EAP      1421          [DET, ADJ, NOUN, AUX, VERB, PUNCT, SPACE]    3  1432   
HPL      1024  [SCONJ, DET, NOUN, VERB, PART, AUX, VERB, ADP,...    1  1024   
MWS      1144  [PRON, VERB, SCONJ, ADP, DET, ADJ, NOUN, ADP, ...    1  1144   

                        
       unique top freq  
Auteur                  
EAP       182  []  820  
HPL       183  []  488  
MWS       138  []  643  

[3 rows x 28 columns]

Comptage des occurences des mots :

In [10]:
df['clean_text']=str()
for i, row in df.iterrows():
    row['clean_text']=' '.join(row['lemma'])
    

df['clean_text'] = df.clean_text.replace("[PRON\s\W]", " ",regex=True)
df['clean_text'] = df.clean_text.replace(' +',' ',regex=True)
df['clean_text'] = df.clean_text.replace('^ ','',regex=True)

#df = pd.get_dummies(df, columns=['Auteur'])
df['Auteur_number']=df['Auteur'].map({'EAP': 1, 'HPL':2, 'MWS':3})


In [11]:
df_finale=df[['Auteur_number','clean_text','ent']]

In [19]:
y = df['Auteur_number']
#X=df_finale.drop(['Auteur_number'], axis=1)
df_finale["clean_text"]= df_finale["clean_text"].astype(str) + df_finale["ent"].astype(str)
X= df_finale["clean_text"]
print(X)
print(y)

0       Idris be well content with this resolve of ['P...
1       be faint even fainter than the hateful moderni...
2       above all burn to know the incident of own pas...
3       may see perhaps one or two point with unusual ...
4       all obey the Lord rotector of die England all ...
                              ...                        
3595    countenance be paint with the hue of ill healt...
3596    must have be merely the association of any odd...
3597    danger meet unflinchingly crime commit unmoved []
3598    as this state of wretchedness become more conf...
3599    be the pride of ancestry the patrician spirit ...
Name: clean_text, Length: 3600, dtype: object
0       3
1       2
2       1
3       1
4       3
       ..
3595    3
3596    2
3597    2
3598    3
3599    3
Name: Auteur_number, Length: 3600, dtype: int64


b:\agilytic\nlp\nlp_agilytic\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
auteurs=['EAP','HPL', 'MWS']

In [21]:
#Test avec une Multinomial regression
nb = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
nb.fit(X_train, y_train)
y_pred_NB = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred_NB, y_test))
print(classification_report(y_test, y_pred_NB,target_names=auteurs))

accuracy 0.687037037037037
              precision    recall  f1-score   support

         EAP       0.58      0.92      0.71       412
         HPL       0.92      0.41      0.57       318
         MWS       0.82      0.66      0.73       350

    accuracy                           0.69      1080
   macro avg       0.77      0.66      0.67      1080
weighted avg       0.76      0.69      0.68      1080



In [22]:
#Test avec GradientBoost
gb = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('gboost', GradientBoostingClassifier(max_depth=6,   n_estimators=500, random_state=2)),])
gb.fit(X_train,y_train)
y_pred_boost = gb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred_boost, y_test))
print(classification_report(y_test, y_pred_boost,target_names=auteurs))

accuracy 0.6601851851851852
              precision    recall  f1-score   support

         EAP       0.62      0.80      0.70       412
         HPL       0.69      0.54      0.60       318
         MWS       0.71      0.60      0.65       350

    accuracy                           0.66      1080
   macro avg       0.67      0.65      0.65      1080
weighted avg       0.67      0.66      0.66      1080



In [38]:
#Test avec KNN
kn = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('knn', KNeighborsClassifier(n_neighbors=40)),])
kn.fit(X_train,y_train)
y_pred_kn = kn.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred_kn, y_test))
print(classification_report(y_test, y_pred_kn,target_names=auteurs))

TypeError: __init__() got an unexpected keyword argument 'random_state'

In [37]:
#Test avec SVM
sv = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('SVM', SGDClassifier()),])
sv.fit(X_train,y_train)
y_pred_svm = sv.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred_svm, y_test))
print(classification_report(y_test, y_pred_svm,target_names=auteurs))

accuracy 0.7379629629629629
              precision    recall  f1-score   support

         EAP       0.72      0.73      0.72       412
         HPL       0.75      0.72      0.74       318
         MWS       0.74      0.77      0.75       350

    accuracy                           0.74      1080
   macro avg       0.74      0.74      0.74      1080
weighted avg       0.74      0.74      0.74      1080



In [25]:
#Test avec Logistic regression
lr = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('lrg', LogisticRegression()),])
lr.fit(X_train,y_train)
y_pred_lrg = lr.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred_lrg, y_test))
print(classification_report(y_test, y_pred_lrg,target_names=auteurs))

accuracy 0.7148148148148148
              precision    recall  f1-score   support

         EAP       0.65      0.80      0.72       412
         HPL       0.77      0.60      0.68       318
         MWS       0.78      0.71      0.75       350

    accuracy                           0.71      1080
   macro avg       0.73      0.71      0.71      1080
weighted avg       0.73      0.71      0.71      1080



In [34]:
#Test avec RandomForest
from sklearn.ensemble import RandomForestClassifier
rf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('rfo', RandomForestClassifier(n_estimators=500, min_samples_leaf=0.00001)),])
rf.fit(X_train,y_train)
y_pred_rfo = rf.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred_rfo, y_test))
print(classification_report(y_test, y_pred_rfo,target_names=auteurs))

accuracy 0.6175925925925926
              precision    recall  f1-score   support

         EAP       0.54      0.91      0.68       412
         HPL       0.74      0.36      0.48       318
         MWS       0.77      0.51      0.61       350

    accuracy                           0.62      1080
   macro avg       0.68      0.59      0.59      1080
weighted avg       0.67      0.62      0.60      1080

